In [1]:
import pandas as pd
from itertools import chain
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from collections import Counter

In [2]:
data_dir = '../synonyms_from_category/data/'
dt = '20190130'

In [7]:
df_synonyms = pd.read_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/candidate_synonyms_final_190130.tsv', sep='\t', names=['word', 'synonyms'])


In [10]:
df_users = pd.read_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/cate_WE_user_dict_190130.txt', names=['word'])